In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import urllib.request
import json

In [ ]:
file1 = open("best.txt","r")
links=(file1.readlines()) 
for i in range(len(links)):
    links[i]=links[i].replace("\n","")
print(len(links),type(links[0]))

In [ ]:
output=[]
for i in range(len(links)):
    d={}
    Web_url = links[i]
    r = requests.get(Web_url)
    soup = BeautifulSoup(r.content, 'html5lib')
    meta=soup.findAll('meta',{'property':['og:description','og:title','og:url']})
    d['content']=meta[0].get('content')
    d['title']=meta[1].get('content')
    url=meta[2].get('content')
    url=list(url.split('/'))
    d['url']=url[-2]
    print(d)
    output.append(d)
json_dump=json.dumps(output)
file_name="title-description.json"
with open(file_name, "w") as outfile:
    outfile.write(json_dump)

In [ ]:
for i in range(len(links)):
    Web_url = links[i]
    print(Web_url)
    r = requests.get(Web_url)
    webstoryno=i+1
    os.makedirs("webstory-"+str(webstoryno))
    soup = BeautifulSoup(r.content, 'html5lib')
    stories=soup.findAll('amp-story-page')
    output=[]
    items=0
    for st in stories:
        story={}
        story['url']=Web_url.split('/')[-2]
        headings=(st.findAll('h1'))
        subheadings=(st.findAll('h2'))
        paras=(st.findAll('p'))
        story['heading']=[]
        for heading in headings:
            story['heading'].append(heading.text)
        story['subheading']=[]
        for subheading in subheadings:
            story['subheading'].append(subheading.text)
        story['para']=[]
        for para in paras:
            story['para'].append(para.text)
        image=st.find('amp-img')
        poster=st.find('amp-video')
        video=st.findAll('source')
        story['image']=""
        story['poster']=""
        story['video']=""
        if image:
            items+=1
            src=image['src']
            img_src=src
            src=list(src.split('/'))
            src="webstory-"+str(webstoryno)+"-"+src[-1]
            story['image']=src
            src="webstory-"+str(webstoryno)+'/'+src
            try:
                urllib.request.urlretrieve(img_src,src)
            except:
                print(img_src,src,webstoryno)
        if poster:
            items+=1
            src=poster['poster']
            post_src=src
            src=list(src.split('/'))
            src="webstory-"+str(webstoryno)+"-"+src[-1]
            story['poster']=src
            src="webstory-"+str(webstoryno)+'/'+src
            try:
                urllib.request.urlretrieve(post_src,src)
            except:
                print(post_src,src,webstoryno)
        if video:
            items+=1
            src=video[-1]['src']
            video_src=src
            src=list(src.split('/'))
            src="webstory-"+str(webstoryno)+"-"+src[-1]
            story['video']=src
            src="webstory-"+str(webstoryno)+'/'+src
            try:
                urllib.request.urlretrieve(video_src,src)
            except:
                print(video_src,src,webstoryno)
        output.append(story)
    print("No of items:",items+1)
    json_dump=json.dumps(output)
    file_name="webstory-"+str(webstoryno)+'/'+"webstory-"+str(webstoryno)+".json"
    with open(file_name, "w") as outfile:
        outfile.write(json_dump)